In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

In [2]:

# Load and preprocess the dataset
train = pd.read_csv("engtamilTrain.csv").drop(["Unnamed: 0"], axis=1)
english_sentences = train["en"].head(1000)
tamil_sentences = train["ta"].head(1000)

In [3]:
english_sentences

0      MMA vice president Qazi Hussain Ahmad declared...
1      Information has surfaced in recent years sugge...
2      And Azor begat Sadoc; and Sadoc begat Achim; a...
3      She says she knows what is going on, but can d...
4      And be it indeed that I have erred, my error r...
                             ...                        
995    A face that stays on in people's mind is impor...
996    According to a report distributed by the Pales...
997    In a fundraising letter to big contributors, B...
998    It is entirely appropriate that outraged prote...
999    Against him there exists nothing, and he knows...
Name: en, Length: 1000, dtype: object

In [4]:
tamil_sentences

0      MMA கட்சியின் துணைத்தலைவர் க்வாஸி ஹுசேன் அகமத்...
1      சமீபகாலத்தில் சில தகவல்கள் யூலியஸ் ரோசன்பேர்க...
2      ஆசோர் சாதோக்கைப் பெற்றான்; சாதோக்கு ஆகீமைப் பெ...
3      என்ன நடக்கிறது என்பது தமக்கு தெரியும் என்றும் ...
4      நான் தப்பிநடந்தது மெய்யானாலும், என் தப்பிதம் எ...
                             ...                        
995    ஒரு நடிகருக்கோ, நடிகைக்கோ மனதில் பதியக்கூடிய ம...
996    ஒரு அகதி முகாமின் ஒரு தெருவில் 30 பேர் கொல்லப...
997    புஷ் பிரச்சார பொது வக்கீலான Tom Josejiak விடு...
998    ஆத்திரம் கொண்ட ஆர்ப்பாட்டக்காரர்கள் CNN, A.B....
999    அவருக்கு எதிராக ஒன்றும் இல்லை; அதை அவர் நன்கு ...
Name: ta, Length: 1000, dtype: object

In [5]:
# Function to add <SOS> and <EOS> tokens to each sentence
def addSosEos(seriesSentence):
    sos_token, eos_token = "<SOS>", "<EOS>"
    return [f"{sos_token} {sentence} {eos_token}" for sentence in seriesSentence]

In [6]:
# Apply the function to add tokens
english_sent_SE = addSosEos(english_sentences)
tamil_sent_SE = addSosEos(tamil_sentences)

In [7]:
english_sent_SE

["<SOS> MMA vice president Qazi Hussain Ahmad declared last month: 'We are not extremists.\n <EOS>",
 '<SOS> Information has surfaced in recent years suggesting that Julius Rosenberg was involved in passing some form of intelligence to Soviet officials during the Second World War.\n <EOS>',
 '<SOS> And Azor begat Sadoc; and Sadoc begat Achim; and Achim begat Eliud;\n <EOS>',
 '<SOS> She says she knows what is going on, but can do nothing about it.\n <EOS>',
 '<SOS> And be it indeed that I have erred, my error remains with myself.\n <EOS>',
 '<SOS> Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality.\n <EOS>',
 '<SOS> These include the British Tamil Forum, La Maison du Tamil Eelam (France), the Canadian Tamil Congress, and the Swiss Tamil Forum.\n <EOS>',
 "<SOS> Vijay accompanied with his wife and daughter enjoyed the film 'Anjathey'.\n <EOS>",
 '<SOS> Both Musharraf and Vajpayee have exp

In [8]:
tamil_sent_SE

['<SOS> MMA கட்சியின் துணைத்தலைவர் க்வாஸி ஹுசேன் அகமத் சென்ற மாதம் பின்வருமாறு அறிவித்தார்: ``நாங்கள் தீவிரவாதிகள் அல்ல.\n <EOS>',
 '<SOS> சமீபகாலத்தில் சில தகவல்கள் யூலியஸ் ரோசன்பேர்க் ஒரு வித உளவுச்செய்தியை சோவியத் அதிகாரிகளுக்கு இரண்டாம் உலகப்போரின்போது அனுப்பியதில் சம்பந்தப்பட்டு இருந்ததாக வெளிவந்துள்ளன.\n <EOS>',
 '<SOS> ஆசோர் சாதோக்கைப் பெற்றான்; சாதோக்கு ஆகீமைப் பெற்றான்; ஆகீம் எலியூதைப் பெற்றான்;\n <EOS>',
 '<SOS> என்ன நடக்கிறது என்பது தமக்கு தெரியும் என்றும் ஆனால், தம்மால் எதுவும் செய்யமுடியாது என்றும் கடிதம் எழுதியிருந்தார்.\n <EOS>',
 '<SOS> நான் தப்பிநடந்தது மெய்யானாலும், என் தப்பிதம் என்னோடேதான் இருக்கிறது\n <EOS>',
 '<SOS> டால்ரிம்பிளினுடைய அறிவுஜீவித் தொடுவானத்திற்கு அப்பால் எவ்வளவோ தொலைவில் இருந்தன\n <EOS>',
 '<SOS> இந்த அமைப்புக்களில் British Tamil Forum, La Maison du Tamil Eelam (France), the Canadian Tamil Congress, Swiss Tamil Forum<span lang="DE"> ஆகியவை அடங்கும்.\n <EOS>',
 "<SOS> தனது மனைவி, மகளுடன் 'அஞ்சாதே' படத்தை ரசித்தார் விஜய்.\n <EOS>",
 '<SOS> முஷார

In [9]:
# Tokenize the English and Tamil sentences
english_tokenizer = Tokenizer(filters="")
english_tokenizer.fit_on_texts(english_sent_SE)
english_tokenizer





In [10]:
english_vocab_size = len(english_tokenizer.word_index) + 1
english_vocab_size

6979

In [11]:
english_sequences = english_tokenizer.texts_to_sequences(english_sent_SE)

In [12]:
english_sequences

[[2, 2036, 2037, 224, 2038, 1113, 2039, 285, 57, 2040, 445, 24, 23, 2041, 3],
 [2,
  762,
  18,
  2042,
  7,
  567,
  143,
  2043,
  9,
  2044,
  2045,
  15,
  568,
  7,
  2046,
  126,
  1114,
  4,
  340,
  6,
  446,
  447,
  184,
  1,
  341,
  105,
  342,
  3],
 [2, 5, 2047, 448, 2048, 5, 2049, 448, 2050, 5, 2051, 448, 2052, 3],
 [2, 86, 449, 86, 343, 106, 10, 344, 1115, 37, 91, 127, 450, 79, 225, 3],
 [2, 5, 20, 19, 451, 9, 33, 22, 2053, 59, 2054, 763, 17, 2055, 3],
 [2,
  1116,
  1,
  1117,
  2056,
  6,
  569,
  35,
  41,
  172,
  1,
  67,
  452,
  4,
  1,
  1118,
  226,
  5,
  1119,
  16,
  2057,
  172,
  1,
  2058,
  4,
  2059,
  3],
 [2,
  53,
  764,
  1,
  345,
  92,
  2060,
  2061,
  2062,
  1120,
  92,
  765,
  2063,
  1,
  2064,
  92,
  2065,
  5,
  1,
  2066,
  92,
  1121,
  3],
 [2, 766, 2067, 17, 21, 1122, 5, 453, 1123, 1, 78, 2068, 3],
 [2,
  95,
  767,
  5,
  1124,
  22,
  768,
  1,
  346,
  49,
  2069,
  6,
  2070,
  128,
  769,
  26,
  34,
  2071,
  25,
  570,
  5,
  6

In [13]:
tamil_tokenizer = Tokenizer(filters="")
tamil_tokenizer.fit_on_texts(tamil_sent_SE)
tamil_vocab_size = len(tamil_tokenizer.word_index) + 1
tamil_sequences = tamil_tokenizer.texts_to_sequences(tamil_sent_SE)

In [14]:
# Define maximum sequence lengths
max_input_seq_length = 20
max_output_seq_length = 20

In [15]:

# Pad sequences to a fixed length
input_sequences = pad_sequences(english_sequences, maxlen=max_input_seq_length, padding='post')
output_sequences = pad_sequences(tamil_sequences, maxlen=max_output_seq_length, padding='post')

In [16]:
input_sequences[0]

array([   2, 2036, 2037,  224, 2038, 1113, 2039,  285,   57, 2040,  445,
         24,   23, 2041,    3,    0,    0,    0,    0,    0])

In [17]:
output_sequences[0]

array([   1, 1836,   44, 1837, 1838, 1839, 1840,  148,   78,  884, 1841,
       1842,  885,  282,    2,    0,    0,    0,    0,    0])

In [18]:
# Prepare decoder input and output sequences for training
decoder_input_sequences = np.zeros_like(output_sequences)
decoder_input_sequences[:, 1:] = output_sequences[:, :-1]
decoder_input_sequences[:, 0] = tamil_tokenizer.word_index.get('<SOS>', 0)

decoder_output_sequences = tf.keras.utils.to_categorical(output_sequences, num_classes=tamil_vocab_size)

In [19]:
decoder_output_sequences[0]

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [20]:
# Load pre-trained Word2Vec models
eng_model = Word2Vec.load('engmodel.bin')
tam_model = Word2Vec.load('tammodel.bin')

In [21]:
# Function to create embedding matrix from Word2Vec model
def create_embedding_matrix(word2vec_model, tokenizer, vocab_size):
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
    return embedding_matrix

In [22]:
eng_model

In [23]:
english_tokenizer

In [24]:
english_vocab_size

6979

In [25]:
# Create embedding matrices for both languages
eng_embedding_matrix = create_embedding_matrix(eng_model, english_tokenizer, english_vocab_size)
tam_embedding_matrix = create_embedding_matrix(tam_model, tamil_tokenizer, tamil_vocab_size)

In [26]:
eng_embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
# Function to create the Seq2Seq model
def create_seq2seq_model(input_vocab_size, output_vocab_size, input_seq_length, output_seq_length, hidden_units, eng_embedding_matrix, tam_embedding_matrix):
    # Encoder
    encoder_inputs = Input(shape=(input_seq_length,))
    encoder_embedding = Embedding(input_vocab_size, hidden_units, weights=[eng_embedding_matrix], trainable=False)(encoder_inputs)
    encoder_lstm, encoder_state_h, encoder_state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_length,))
    decoder_embedding = Embedding(output_vocab_size, hidden_units, weights=[tam_embedding_matrix], trainable=False)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

In [28]:
# Create and compile the Seq2Seq model
model = create_seq2seq_model(english_vocab_size, tamil_vocab_size, max_input_seq_length, max_output_seq_length, 100, eng_embedding_matrix, tam_embedding_matrix)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Train the model
batch_size = 32
epochs = 500
model.fit([input_sequences, output_sequences], decoder_output_sequences, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - accuracy: 0.2228 - loss: 9.1194 - val_accuracy: 0.2717 - val_loss: 8.3275
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.2755 - loss: 7.6880 - val_accuracy: 0.2667 - val_loss: 7.5263
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.2715 - loss: 6.5730 - val_accuracy: 0.2717 - val_loss: 7.5061
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.2597 - loss: 6.5113 - val_accuracy: 0.2722 - val_loss: 7.6059
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.2679 - loss: 6.3574 - val_accuracy: 0.2722 - val_loss: 7.6662
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.2699 - loss: 6.2888 - val_accuracy: 0.2715 - val_loss: 7.5388
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.2752 - loss: 6.1840 - val_accuracy: 0.2720 - val_loss: 7.5393
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.2553 - loss: 6.2905 - val_accur

In [30]:
# Preprocessing and predicting
input_sentence = "<SOS> Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality <EOS>"


In [31]:

# Convert the input sentence to sequence and pad it
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_length, padding='post')

In [32]:
input_sequence

array([[ 569,   35,   41,  172,    1,   67,  452,    4,    1, 1118,  226,
           5, 1119,   16, 2057,  172,    1, 2058,    4,    3]])

In [33]:
# Generate predictions
predictions = model.predict([input_sequence, np.zeros((1, max_output_seq_length))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 653ms/step


In [34]:
predictions

array([[[4.85001551e-03, 4.68601614e-01, 6.83055632e-03, ...,
         5.01609136e-11, 4.10023508e-11, 4.12120095e-11],
        [3.33048287e-04, 6.80165176e-05, 1.42812321e-03, ...,
         5.79025883e-10, 5.28762978e-10, 5.20343935e-10],
        [4.45979495e-05, 1.11430006e-06, 2.87042116e-04, ...,
         1.77326498e-09, 1.68188385e-09, 1.83374671e-09],
        ...,
        [9.88464534e-01, 3.73382408e-10, 1.11269355e-02, ...,
         2.96680432e-13, 3.72327169e-13, 3.55200515e-13],
        [9.98560011e-01, 1.54561627e-10, 1.35702838e-03, ...,
         5.31495285e-14, 6.36453984e-14, 6.11375235e-14],
        [9.99147892e-01, 9.81351320e-11, 8.29990604e-04, ...,
         1.44010814e-14, 1.62971003e-14, 1.59915806e-14]]], dtype=float32)

In [35]:
# Convert predictions to tokens and decode
predicted_tokens = np.argmax(predictions, axis=-1)[0]
tamil_index_word = {i: w for w, i in tamil_tokenizer.word_index.items()}
decoded_sentence = [tamil_index_word.get(token, '<unk>') for token in predicted_tokens if token != 0 and token != tamil_tokenizer.word_index.get('<EOS>', 0)]

In [36]:
# Join the words to form the decoded statement
decoded_statement = ' '.join(decoded_sentence)

In [37]:

# Print the decoded statement
print(decoded_statement)

<sos> அப்பொழுது அடிமைப்பட்டிருந்த பிதாக்களின் அப்பால் எங்களை புறப்படப்பண்ணின இருந்தன
 குண்டுவீச்சில் வலியுறுத்தியதை குடும்பத்தை ஈராக்கிய <eos>


In [38]:
# Print additional details for debugging
print("Predictions:", predictions)

Predictions: [[[4.85001551e-03 4.68601614e-01 6.83055632e-03 ... 5.01609136e-11
   4.10023508e-11 4.12120095e-11]
  [3.33048287e-04 6.80165176e-05 1.42812321e-03 ... 5.79025883e-10
   5.28762978e-10 5.20343935e-10]
  [4.45979495e-05 1.11430006e-06 2.87042116e-04 ... 1.77326498e-09
   1.68188385e-09 1.83374671e-09]
  ...
  [9.88464534e-01 3.73382408e-10 1.11269355e-02 ... 2.96680432e-13
   3.72327169e-13 3.55200515e-13]
  [9.98560011e-01 1.54561627e-10 1.35702838e-03 ... 5.31495285e-14
   6.36453984e-14 6.11375235e-14]
  [9.99147892e-01 9.81351320e-11 8.29990604e-04 ... 1.44010814e-14
   1.62971003e-14 1.59915806e-14]]]


In [39]:
print("Predicted tokens:", predicted_tokens)

Predicted tokens: [   1   94 5882 1058 1873 3879 5886 1875 2781 1531 2782  794    2    0
    0    0    0    0    0    0]


In [40]:


print("Input sequence:", input_sequence)


Input sequence: [[ 569   35   41  172    1   67  452    4    1 1118  226    5 1119   16
  2057  172    1 2058    4    3]]
